In [1]:
!git clone https://github.com/Mr-Patty/bimodal-emotion-recognition
# !pip install soundfile

Cloning into 'bimodal-emotion-recognition'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 103 (delta 44), reused 71 (delta 15), pack-reused 0
Receiving objects: 100% (103/103), 3.12 MiB | 15.82 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!cp drive/'My Drive'/EmotionRecognition/bimodal-emotion-recognition.tar.gz bimodal-emotion-recognition/
%cd bimodal-emotion-recognition
!cp ../drive/'My Drive'/EmotionRecognition/df_prep.csv .
!tar -C . -xzf bimodal-emotion-recognition.tar.gz
!rm bimodal-emotion-recognition.tar.gz

/content/bimodal-emotion-recognition


In [1]:
%cd bimodal-emotion-recognition

/content/bimodal-emotion-recognition


In [2]:
!ls

AudioModel.ipynb  diploma.ipynb        preprocess.py	Train_data
chkpt		  experiment.ipynb     processing.py	utils
config		  logs		       README.md	utils.py
datasets	  models.py	       test.png
df_prep.csv	  Preprocessing.ipynb  TextModel.ipynb


In [2]:
import os
import glob
import torch
import random
import numpy as np
from torch.utils.data import Dataset, DataLoader
from utils.hparams import HParam
from datasets.dataloader import create_dataloader, create_dataset

from utils.utils import read_wav_np
import wave
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import math
import librosa
import json
import wave
import sys
import pickle
import sklearn

import urllib.request

import librosa.display
import scipy, matplotlib.pyplot as plt, IPython.display as ipd

from scipy.io import wavfile

import librosa.display
# import soundfile as sf

import torch
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import os
import matplotlib.style as ms
from tqdm import tqdm
import random
import logging
import time

from utils.train import *
# from models import *
ms.use('seaborn-muted')
%matplotlib inline

In [3]:
hp = HParam('config/default.yaml')
with open('config/default.yaml', 'r') as f:
    hp_str = ''.join(f.readlines())

In [4]:
pt_dir = os.path.join(hp.log.chkpt_dir, 'AudioModel')
log_dir = os.path.join(hp.log.log_dir, 'AudioModel')
os.makedirs(pt_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

In [5]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(os.path.join(log_dir,
            '%s-%d.log' % ('AudioModel', time.time()))),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

In [6]:
assert hp.audio.hop_length == 256, \
    'hp.audio.hop_length must be equal to 256, got %d' % hp.audio.hop_length
assert hp.data.train != '' \
    'hp.data.train and hp.data.validation can\'t be empty: please fix %s' % 'config/default.yaml'

In [9]:
trainset = create_dataset(hp, 'df_prep.csv', True)
train_loader = DataLoader(trainset, num_workers=1, shuffle=False,
                              batch_size=1)

In [ ]:
min_n = 0
for i in tqdm(train_loader):
    x = i[0]
#     print(x.shape)
#     if x.shape[1] > min_n:
#         min_n = x.shape[1]
# min_n

In [ ]:
class AttentionRNNModel(torch.nn.Module):
    def __init__(self, batch_size, output_size, hidden_size, embedding_length, bidirectional=False, num_layers=1, dropout=0.5):
        super(AttentionRNNModel, self).__init__()

        """
        Arguments
        ---------
        batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
        output_size : 2 = (pos, neg)
        hidden_sie : Size of the hidden_state of the LSTM
        vocab_size : Size of the vocabulary containing unique words
        embedding_length : Embeddding dimension of GloVe word embeddings
        weights : Pre-trained GloVe word_embeddings which we will use to create our word_embedding look-up table 

        --------

        """

        self.batch_size = batch_size
        self.output_size = output_size
        self.hidden_size = hidden_size
#         self.vocab_size = vocab_size
        self.embedding_length = embedding_length
        self.bidirectional = bidirectional
        self.num_layers = num_layers

#         self.word_embeddings = nn.Embedding(vocab_size, embedding_length)
#         self.word_embeddings.weights = nn.Parameter(weights, requires_grad=False)
        self.lstm = nn.LSTM(embedding_length, hidden_size, bidirectional=self.bidirectional, num_layers=self.num_layers)
        self.label = nn.Linear(hidden_size * 2 if self.bidirectional else hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)
        #self.attn_fc_layer = nn.Linear()

    def attention_net(self, lstm_output, final_state):

        """ 
        Now we will incorporate Attention mechanism in our LSTM model. In this new model, we will use attention to compute soft alignment score corresponding
        between each of the hidden_state and the last hidden_state of the LSTM. We will be using torch.bmm for the batch matrix multiplication.

        Arguments
        ---------

        lstm_output : Final output of the LSTM which contains hidden layer outputs for each sequence.
        final_state : Final time-step hidden state (h_n) of the LSTM

        ---------
    
        Returns : It performs attention mechanism by first computing weights for each of the sequence present in lstm_output and and then finally computing the
                  new hidden state.

        Tensor Size :
                hidden.size() = (batch_size, hidden_size)
                attn_weights.size() = (batch_size, num_seq)
                soft_attn_weights.size() = (batch_size, num_seq)
                new_hidden_state.size() = (batch_size, hidden_size)

        """

        hidden = final_state.squeeze(0)
        attn_weights = torch.bmm(lstm_output, hidden.unsqueeze(2)).squeeze(2)
        soft_attn_weights = F.softmax(attn_weights, 1)
        new_hidden_state = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2)

        return new_hidden_state

    def forward(self, input, batch_size=None):

        """ 
        Parameters
        ----------
        input_sentence: input_sentence of shape = (batch_size, num_sequences)
        batch_size : default = None. Used only for prediction on a single sentence after training (batch_size = 1)
    
        Returns
        -------
        Output of the linear layer containing logits for pos & neg class which receives its input as the new_hidden_state which is basically the output of the Attention network.
        final_output.shape = (batch_size, output_size)

        """

#         input = self.word_embeddings(input_sentences)
        input = input.permute(1, 0, 2)
        # h_0 = torch.zeros(1, self.batch_size, self.hidden_size)
        # c_0 = torch.zeros(1, self.batch_size, self.hidden_size)

        output, (final_hidden_state, final_cell_state) = self.lstm(input) # final_hidden_state.size() = (1, batch_size, hidden_size) 
        # output = output.permute(0, 2, 1) # output.size() = (batch_size, num_seq, hidden_size)

        # attn_output = self.attention_net(output, final_hidden_state)
        final_hidden_state = self.dropout(torch.cat((final_hidden_state[-2,:,:], final_hidden_state[-1,:,:]), dim = 1))
        logits = self.label(final_hidden_state)

        return logits

In [ ]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)
# train_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size=hp.train.batch_size, shuffle=True,
#             num_workers=hp.train.num_workers, pin_memory=True, drop_last=False)
# model = model.to(device)
# loss_func = nn.CrossEntropyLoss()
# for x, y in train_loader:
#     # y_pred = model(x)
#     x = x.to(device)
#     y = y.to(device)
#     y_pred = model(x)
#     loss = loss_func(y_pred, y)
#     print(y_pred.shape)
#     print(y)
#     # print(x.shape)
#     break

In [ ]:
Batch = 1
model = AttentionRNNModel(batch_size=Batch, output_size=7, hidden_size=32, embedding_length=80, bidirectional=True)
trainset = create_dataset(hp, 'df_prep.csv', True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_loss, val_loss, train_acces, test_acces, best_model = train(model, trainset, device, batch_size=Batch, n_epoch=50, validation_split=.3)

In [ ]:
!cp audio-model.pt drive/'My Drive'/EmotionRecognition/